In [22]:
import os
import re
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession

def init_spark(app_name: str):
  spark = SparkSession.builder.appName(app_name).getOrCreate()
  sc = spark.sparkContext
  return spark, sc

spark, sc = init_spark('demo')
sc

<SparkContext master=local[*] appName=demo>

In [23]:
data_file = '500k_daily_prog_data.csv'
view_rdd = sc.textFile(data_file)
print('Type:', type(view_rdd))
print('Count (rows):', view_rdd.count())

Type: <class 'pyspark.rdd.RDD'>
Count (rows): 499528


In [24]:
view_rdd.take(1)

['prog_code,title,genre,air_date,air_time,Duration']

In [25]:
# Split by comma:
csv_rdd = view_rdd.map(lambda row: row.split(','))
csv_rdd.take(2)

[['prog_code', 'title', 'genre', 'air_date', 'air_time', 'Duration'],
 ['EP000002190577', 'Almanac', 'Newsmagazine', '20151219', '070000', '60.0']]

In [30]:
# Split to header and data:
header = csv_rdd.first()
print(header)
data_rdd = csv_rdd.filter(lambda row: row != header)
data_rdd.first()

['prog_code', 'title', 'genre', 'air_date', 'air_time', 'Duration']


['EP000002190577', 'Almanac', 'Newsmagazine', '20151219', '070000', '60.0']

In [31]:
import re

def clean_strings(string_list):
    cleaned_list = []
    for s in string_list:
        cleaned_list.append(re.sub(r'^"|"$', '', s))
    return cleaned_list

data_rdd.map(lambda row: (row[1], clean_strings((row[2:-3])), row[-2])).take(5)

[('Almanac', ['Newsmagazine'], '070000'),
 ('Are You Being Served?', ['Sitcom'], '143000'),
 ('Austin City Limits', ['Music'], '043000'),
 ('The Big Valley', ['Drama', 'Western'], '090000'),
 ('Ciao Italia', ['Cooking'], '120000')]

In [9]:
popular_genres = ['Talk', 'Politics', 'Spanish', 'Community', 'Martial arts']
prime_time = data_rdd.filter(lambda row: (70000<=int(row[4])<90000) and row[2] in popular_genres)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 7) (DESKTOP-ALFRKTH.mshome.net executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\yogev\anaconda3\envs\Distibuted\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "C:\Users\yogev\anaconda3\envs\Distibuted\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 611, in process
  File "C:\Users\yogev\anaconda3\envs\Distibuted\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\yogev\anaconda3\envs\Distibuted\lib\site-packages\pyspark\rdd.py", line 1563, in takeUpToNumLeft
    except StopIteration:
  File